In [1]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def print_transcript_dist(transcript_dist, gene_name):
    plt.gca().clear()
    axes = plt.gca()
    axes.set_ylim([0, max(transcript_dist) + 1])
    plt.plot(transcript_dist)
    plt.ylabel("read coverage")
    plt.xlabel("transcript location")
    plt.title(gene_name)
    plt.savefig("dropseq_dists_100/" + gene_name + ".png")

    return

In [3]:
#read in the index 
indexes = []
with open("transcript_index.txt") as f_index:
    for line in f_index:
        indexes.append(int(line[:-1]))
print(indexes[:10])

[54, 81, 83, 86, 98, 104, 121, 123, 126, 137]


In [4]:
#read in the start positions
start_positions = []
with open("transcript_start_position.txt") as f_start_position:
    for line in f_start_position:
        start_positions.append(int(line[:-1]))
print(start_positions[:10])

[52726612, 52943019, 52948890, 52972759, 53073004, 53132851, 53254047, 53259900, 53283896, 53363836]


In [5]:
#read in the transcripts
gene_names = []
transcripts = []
with open("Mus_musculus.GRCm38.cdna.all.fa") as f_transcript:
    cur_transcript = ""
    for line in f_transcript:
        if(line[0] == ">"):
            gene_name = line[1:]
            gene_name_arr = gene_name.split()
            gene_name = gene_name_arr[0]
            gene_names.append(gene_name)
            if(cur_transcript != ""):
                transcripts.append(cur_transcript)
                cur_transcript = ""
        else:
            cur_transcript = cur_transcript + line[:-1]
print(transcripts[:10])
print(gene_names[:10])

['ATGGCATAT', 'ATGGCATATCA', 'ATCGGAGGGATACGAG', 'GGGACAGGGGGC', 'GGGACTGGGGGGGC', 'CTAACTGGGAC', 'AGACAGCTCAGGCTAC', 'GAATACCTAC', 'TGGAATACCTAC', 'TCTACTATGGTAACTAC']
['ENSMUST00000196221.1', 'ENSMUST00000179664.1', 'ENSMUST00000177564.1', 'ENSMUST00000178537.1', 'ENSMUST00000178862.1', 'ENSMUST00000179520.1', 'ENSMUST00000179883.1', 'ENSMUST00000195858.1', 'ENSMUST00000179932.1', 'ENSMUST00000180001.1']


In [6]:
f_in = open("reads_for_transcripts.txt")
f_scores = open("dropseq_scores_100.txt", "w+")
f_str = f_in.readline()[1:-2]
reads_per_transcript = re.split('\[\[|\]\]', f_str)
counter = 0
count = 1
transcript_dist = []
for i in range(len(reads_per_transcript)):
    cur_reads = reads_per_transcript[i]
    cur_reads_arr = re.split('\]', cur_reads)
    transcript_scores = []
    
    if(not('A' in cur_reads or 'C' in cur_reads or 'G' in cur_reads or 'T' in cur_reads)):
        counter = counter + len(cur_reads_arr) - 1
        continue
    if(len(cur_reads_arr) < 101):
        counter = counter + 1
        continue
    f_scores.write(">" + gene_names[counter] + "\n")
    for j in range(len(cur_reads_arr)):
        cur_read = cur_reads_arr[j].replace('[', '').replace(',', '').replace('\'', '').replace('\"', '')
        if(len(cur_read) > 2 and cur_read[0] == ' '):
            cur_read = cur_read[1:]
        if(len(cur_read) <= 2):
            continue
        cur_read_arr = cur_read.split(' ')
        read = cur_read_arr[0]
        init_loc = int(cur_read_arr[1])
        end_loc = int(cur_read_arr[2])
        
        index = indexes[counter]
        start_position = start_positions[counter]
        #end location is 
        transcript_read = transcripts[index][init_loc - start_position : end_loc - start_position + 1]
        if(len(transcript_dist) == 0):
            for i in range(len(transcripts[index])):
                transcript_dist.append(0)
        alignments = pairwise2.align.localxx(read, transcript_read)
        if(len(alignments) > 0):
            max_a = alignments[0]
            al1, al2, max_score, begin, end = max_a
        for a in alignments:
            al1, al2, score, begin, end = a
            if(score > max_score):
                max_score = score
                max_a = a
        al1, al2, score, begin, end = max_a
        f_scores.write(str(max_score) + ", " + str(end_loc - init_loc) + ", " + str(len(read)) + "\n")
        
        transcript_scores.append(max_score) 
        transcript_dist_fragment = transcript_dist[init_loc - start_position - 1: end_loc - start_position + 1]
        transcript_dist_fragment = [x+1 for x in transcript_dist_fragment]
        transcript_dist = transcript_dist[:init_loc - start_position] + transcript_dist_fragment + transcript_dist[end_loc - start_position + 1:]
        #print(init_loc - start_position)
        #print(end_loc - start_position + 1)
        
    #if there is stuff in the dist add it
    if(len(transcript_dist) > 0):
        counter = counter + 1
        #print(gene_names[counter-1])
        print_transcript_dist(transcript_dist, gene_names[counter]) #figure out the gene portion
        del transcript_dist[:]
        transcript_dist = []
